# Largest prime factor. 

Given a value n, grab the largest prime factor of that number.

This is a very important concept (think RSA encryption standards) and there are a bunch of ways to tackle this issue.

So far this is the one I've had the most fun on, haha. :) 

<a href= 'https://mathworld.wolfram.com/RSANumber.html'>Excerpt from Wolfram Alpha on RSA / Prime Factorization</a>
    
    Despite widespread belief at the time that the message encoded by RSA-129 would take millions of years to break, it was factored in 1994 using a distributed computation which harnessed networked computers spread around the globe performing a multiple polynomial quadratic sieve (Leutwyler 1994). 
    
    


# Diving in

Researching this it seems I have 3 choices:

1. Fermat's Factorisation
    
        1. This guy is slow, and will only take odd integers to factor. 
        2. There is a better version of this that is called Fermat's with trial division, that is superior, but not so good with large numbers. 
    
2. Quadratic Sieve 

        1. Choose a smoothness bound B. The number π(B), denoting the number of prime numbers less than B, will control both the length of the vectors and the number of vectors needed.
        2. Use sieving to locate π(B) + 1 numbers ai such that bi=(ai2 mod n) is B-smooth.
        3. Factor the bi and generate exponent vectors mod 2 for each one.
        4. Use linear algebra to find a subset of these vectors which add to the zero vector. Multiply the corresponding ai together and give the result mod n the name a; similarly, multiply the bi together which yields a B-smooth square b2.
        5. We are now left with the equality a2=b2 mod n from which we get two square roots of (a2 mod n), one by taking the square root in the integers of b2 namely b, and the other the a computed in step 4.
        6. We now have the desired identity: {\displaystyle (a+b)(a-b)\equiv 0{\pmod {n}}}{\displaystyle (a+b)(a-b)\equiv 0{\pmod {n}}}. Compute the GCD of n with the difference (or sum) of a and b. This produces a factor, although it may be a trivial factor (n or 1). If the factor is trivial, try again with a different linear dependency or different a.
    
3. Pollard's Rho Algorithm

## Took me a while to figure it out, so shout out to <a href='https://stackoverflow.com/questions/23287/algorithm-to-find-largest-prime-factor-of-a-number'> Tryptich </a> in the following SO post for me to get an understanding of how to tackle this problem.


    def prime_factors(n):
        # Returns all the prime factors of a positive integer
        factors = []
        d = 2
        while n > 1:
            print(f"{n} and {d}")
            while n % d == 0:
                print(f"--{n}/{d}")
                factors.append(d)
                # here we divide n by d to get the remaining factor 
                n = n/d
            #increment d for next iteration of loop
            d = d + 1
            # don't need to go up to all numbers,
            # because if n>1 and d^2 greater than n
            # we found the largest prime factor
            if d*d > n:
                if n > 1: factors.append(n)
                break
        return factors


# Fermat's Decomposition 


Let N represent an ODD integer, neither a or b is equal to 1. 
        
            N = a^2 - b^2
            a^2 - N = b^2 
        
        if b^2 is a perfect square then you can stop because you found a proper 
        factorizaation of N, an odd integer. Fermat takes an approach that optimizes
        when these prime numbers are deeply ingrained, or close together. Because 
        a and b can be reduced to the difference of two squares. 
    
    Algorithm steps:
        Start with value a that is the rounded value just bigger than the sqrt(n). 
        Then solve for b^2. 
        If b^2 is a square you found the answer, otherwise increment the a value by 1
        
        
        For example looking at 
            N = a^2-b^2 which becomes b^2 - N = a^2
        And if N = (c-d)(c+d) because of least squares 
            Then whenever b is small, the values are close together.
            We start with small b values, because we are arbitrarily 
            choosing a number that is twice the square root of N
    
    Based on least squares difference of two numbers:
    
    Let N = (a-b)(a+b) = a^2 - b^2,
    
        (a+b)^2 = a^2 + 2ab - b^2
        (a-b)^2 = a^2 - 2ab + b^2
    
    Then subtract LHS,
        
        (a+b)^2 - (a-b)^2 = 4ab which reduces to
        
        [(a+b)/2]^2 - [(a-b)/2]^2 = ab
        
    Well we know the pattern of a^2 - b^2 = (a-b)(a+b), so plug and chug
        
        [(a+b)/2 - (a-b)/2] x [(a+b)/2 + (a-b)/2]  = ab
                b           x          a           = ab
        
        
    So (a+b)^2 - (a-b)^2 = ab = N
    
    While the second LHS square
        [(a-b)/2]^2 represents half the difference squared. 

#### Let's implement Fermat's Approach

In [1]:
import math

def fermats_approach(n):
    """Function to run fermat's decomposition on an odd number."""
    
    a = math.ceil(math.sqrt(n)) 
    b_2 = a**2 - n
    is_square_root = lambda x: 1 if int(math.sqrt(x)+.5)**2==x else 0  #lamda func   
    while not is_square_root(b_2):
        a+=1
        b_2 = a**2 - n
    
    return a-math.sqrt(b_2), a+math.sqrt(b_2)

print(f"Fermat's approach resulted in these factors: {fermats_approach(600851475143)}")

Fermat's approach resulted in these factors: (486847.0, 1234169.0)


In [117]:
%%timeit
import math
import timeit

fermat_time = fermats_approach(600851475143)

61.3 ms ± 883 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
import math

def simple_func(n):
    """
    Just go up to the square root of n
    The above one makes sense now from SO
    """
    a = int(math.sqrt(n))
    d=2
    tmp=[]
    while n>1:
        while n%d==0:
            tmp.append(d)
            n = n/d
        d+=1
        # if the remainder is a perfect square 
        # greater than the value of n b/c inverse relationship
        if d**2 > n:
            tmp.append(int(n))  # append the last value of n to stop
            break
    return max(t)

simple_func(13195)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

#### Let's implement Quadratic Sieve 

My problem...

           1. Choose a smoothness bound B. The number π(B), denoting the number of prime numbers less than B,
           will control both the length of the vectors and the number of vectors needed.


How to choose a smoothness bound?? At first glance this seems arbitrary, but there seems to be some information on doing it effectively here (Thanks <a href='https://math.dartmouth.edu/~carlp/PDF/qs08.pdf'>Dartmouth</a>)

Okay...let's go easier and come back. 

<img src="https://www.reactiongifs.com/r/cf-dafuq.gif"><img>

 ### Okay let's go simpler. 
 
Can I just get all the prime numbers up to N, grab each value say p, and then run from big to small to see if N is divisible by p? 

If so, I'm a winner. 

Also if I just go up to sqrt(N) I will get all primes. 

Because, 

        Assume a x b = N
        if a==b then both are equal to sqrt(N). 
        if a!=b
            Then one must be bigger than the other, because they share
            an inverse relationship: a = N/b
        So start from sqrt(N) and increment up to N? 
            

# Sieve of Eratothenes

<img src="https://upload.wikimedia.org/wikipedia/commons/e/eb/Sieve_of_Eratosthenes.gif"></a>


Basically find all prime values of n from 2 up to sqrt(n) 

If the value is unmarked (we know it's a prime number), if it's marked we know it's a composite number. 

As we cross out values that are composite, we also cross out those that are multiples of those values. 

In [207]:
import math

# I read way too deep into this problem. 
def erato_sieve(N):
    """grab all prime numbers up to N using the sieve of eratothenes"""
    nums = [1 for i in range(0, N)]
    nums[0] = nums[1] = 0  # set 1 and 2 to 0
    for i in range(2, int(math.sqrt(N))):
        j = 2
        print(i)
        for j in range(2, N):
            print(i*j)
            if i*j >= N:
                break
            nums[i*j]=0
    nums = [idx for idx,val in enumerate(nums) if val]
    return nums

What a rabbit hole. To be continued. 

In [210]:
%%timeit
import math
import timeit

primes = erato_sieve(600851475143)


KeyboardInterrupt: 